# (Iterative) Hirshfeld method

In this tutorial, we will introduce how to use the `horton_part` API method to execute (iterative) Hirshfeld partitioning methods.

First, we import the required libraries.

In [1]:
import numpy as np
from gbasis.evals.eval import evaluate_basis
from gbasis.wrappers import from_iodata
from grid import AtomGrid, ExpRTransform, UniformInteger
from iodata import load_one
from setup import prepare_grid_and_dens, print_results

from horton_part import HirshfeldIWPart, HirshfeldWPart, ProAtomDB, ProAtomRecord

np.set_printoptions(precision=3, linewidth=np.inf, suppress=True)

In (iterative) Hirshfeld methods, a database of atomic radial density profiles is required. Therefore, different single-atom DFT or other *ab initio* calculations should be executed. Here, we use the Gaussian package. The checkpoint files, in `fchk` format, are generated using Gaussian tools as well.

Next, the radial atomic density is obtained by computing the spherical average of the atomic density. This process can be applied using the `gbasis` and `grid` packages.

In [2]:
def prepare_record(filename):
    """Prepare molecular grid and density."""
    mol = load_one(filename)

    # Specify the integration grid
    rtf = ExpRTransform(5e-4, 2e1, 120 - 1)
    uniform_grid = UniformInteger(120)
    rgrid = rtf.transform_1d_grid(uniform_grid)

    # Get the spin-summed density matrix
    one_rdm = mol.one_rdms.get("post_scf", mol.one_rdms.get("scf"))
    basis = from_iodata(mol)
    assert len(mol.atnums) == 1
    grid = AtomGrid.from_preset(atnum=mol.atnums[0], preset="fine", rgrid=rgrid)

    basis_grid = evaluate_basis(basis, grid.points)
    rho = np.einsum("ab,bp,ap->p", one_rdm, basis_grid, basis_grid, optimize=True)
    spline = grid.spherical_average(rho)
    radial_rho = spline(rgrid.points)

    record = ProAtomRecord(mol.atnums[0], mol.charge, mol.energy, rgrid, radial_rho)
    return record

In this tutorial, we focus on the water molecule and therefore only need data for hydrogen and oxygen atoms. It should be noted that data for both cations and anions are also used.

In [3]:
h_1 = prepare_record("./data/atoms/001__h_001_q+00/mult02/atom.fchk")
h_2 = prepare_record("./data/atoms/001__h_002_q-01/mult01/atom.fchk")
o_7 = prepare_record("./data/atoms/008__o_007_q+01/mult04/atom.fchk")
o_8 = prepare_record("./data/atoms/008__o_008_q+00/mult03/atom.fchk")
o_9 = prepare_record("./data/atoms/008__o_009_q-01/mult02/atom.fchk")

records = [h_1, h_2, o_7, o_8, o_9]
db = ProAtomDB(records)

Next, we prepare the molecular grid and densities as described in the `Basic Usage` section. It should be noted that the radial grids for atoms in the water molecule should match the atomic radial grids used for the corresponding single atoms in the atomic database calculations.

In [4]:
def prepare_argument_dict(mol, grid, rho):
    """Prepare basic input arguments for all AIM methods."""
    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "proatomdb": db,
    }
    return kwargs

## Hirshfeld method

The Hirshfeld method can be executed using the `HirshfeldWPart` class.

In [5]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
kwargs = prepare_argument_dict(mol, grid, rho)
h = HirshfeldWPart(**prepare_argument_dict(mol, grid, rho))
h.do_all()
print_results(h)

The number of electrons: 10.000003764139395
Coordinates of the atoms 
 [[ 0.     0.     0.224]
 [-0.     1.457 -0.896]
 [-0.    -1.457 -0.896]]
Atomic numbers of the atom 
 [8 1 1]

Information of integral grids.
--------------------------------------------------------------------------------
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
   |-- Radial grid size: 120
   |-

## Iterative Hirshfeld method

The iterative Hirshfeld method can be executed using `HirshfeldIWpart` class.

In [6]:
hi = HirshfeldIWPart(**prepare_argument_dict(mol, grid, rho))
hi.do_all()
print_results(hi)


Information of integral grids.
--------------------------------------------------------------------------------
Grid size of molecular grid: 18460
************************************ Atom 0 ************************************
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 18
          18 18 18 18 18 18 26 26 26 26 38 38 38 38 38 50 50 50 50 50
          86 86 110 110 110 110 170 194 194 434 590 590 434 434 434 302 302 302 194 194
          170 110 110 110 110 110 110 110 110 110 110 110 86 50 50 18 18 18 18 18
          
************************************ Atom 1 ************************************
   |-- Radial grid size: 120
   |-- Angular grid sizes: 
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
          6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 18 18 18 18 18
   